In [ ]:
!pip install pulp

# Integer Programming

Integer programming is a mathematical optimization technique used when some or all of the variables in a linear programming problem must take integer values. When all variables must be either 0 or 1 (binary), we call it Binary Integer Programming.

## Common applications of Integer Programming:

1. **Facility location problems** - deciding where to build facilities
2. **Resource allocation** - assigning limited resources to tasks
3. **Scheduling** - allocating time slots to activities
4. **Transportation and logistics** - optimizing routes and shipments
5. **Manufacturing** - production planning and inventory control
6. **Financial planning** - portfolio optimization with discrete investments

## Example 1: Facility Location Problem

Our company already has factories in two cities (City A and City B) and is considering expanding them. We also want to build a warehouse (only one).

Decision variables:
- Build factory in City A? (x1): Added value: $9 million, capital required: $6 million
- Build factory in City B? (x2): Added value: $5 million, capital required: $3 million
- Build warehouse in City A? (x3): Added value: $6 million, capital required: $5 million
- Build warehouse in City B? (x4): Added value: $4 million, capital required: $2 million

Available capital: $10 million

Additional constraint: A warehouse can only be built in a city if there is also a factory there.




![image](https://github.com/AdoHaha/automation_robotics_lab/blob/master/lab3a_expl.png?raw=1)


In [ ]:
    # Let's visualize the problem:
    #
    #                 City A                  City B
    #                 ------                 ------
    # Factory:     $9M value              $5M value
    #              $6M cost               $3M cost
    #                  |                      |
    #                  v                      v
    # Warehouse:   $6M value              $4M value
    #              $5M cost               $2M cost
    #
    # Total capital available: $10M
    # Must choose max 1 warehouse, and only where factory exists

In [ ]:
import pulp
from pulp import *

# Create the model
prob = LpProblem("Facility_Location_Problem", LpMaximize)

In [ ]:
# Decision variables (binary: 0 or 1)
x1 = LpVariable("Factory_CityA", cat=LpBinary)
x2 = LpVariable("Factory_CityB", cat=LpBinary)
x3 = LpVariable("Warehouse_CityA", cat=LpBinary)
x4 = LpVariable("Warehouse_CityB", cat=LpBinary)

In [ ]:
# Objective function: maximize total value
prob += 9*x1 + 5*x2 + 6*x3 + 4*x4, "Total_Added_Value"

In [ ]:
# Constraints
# 1. Capital constraint
prob += 6*x1 + 3*x2 + 5*x3 + 2*x4 <= 10, "Available_Capital"

# 2. Only one warehouse
prob += x3 + x4 <= 1, "Maximum_One_Warehouse"

# 3. Warehouse can only be built if factory exists in same city
prob += x1 - x3 >= 0, "CityA_Warehouse_Requires_Factory"
prob += x2 - x4 >= 0, "CityB_Warehouse_Requires_Factory"

In [ ]:
# Helper function to print the solution
def print_solution(p):
    p.solve()
    print("Status:", LpStatus[p.status])
    for v in p.variables():
        print(v.name, "=", v.varValue)
    print("Objective value =", value(p.objective))

In [ ]:
# Solve the problem
print_solution(prob)

## Binary Problems

Binary integer programming is used for "yes/no" decision problems such as:
- Which route to choose
- Which truck to use
- Whether to make an investment

We can also model more complex logical conditions using binary variables.

## Example 2: Modeling Logical Constraints

Sometimes we need to model that ONE of two constraints must be satisfied, but not necessarily both.

For example, maximize x + y where 0 ≤ x ≤ 10, 0 ≤ y ≤ 10, and EITHER (x + y ≤ 3) OR (3y + x ≤ 3).

We can use a binary variable z to implement this logical OR:

In [ ]:
prob2 = LpProblem("Alternative_Constraints_Problem", LpMaximize)

# Continuous variables
x = LpVariable("x", 0, 10, cat=LpContinuous)
y = LpVariable("y", 0, 10, cat=LpContinuous)

# Binary variable to implement logical OR
z = LpVariable("ignore_first_constraint", cat=LpBinary)

# Objective function
prob2 += x + y, "Simple_Sum"

# If z=1, first constraint is relaxed (ignored)
# If z=0, second constraint is relaxed (ignored)
M = 10000  # A very large number
prob2 += x + y <= 3 + M*z, "first_constraint"
prob2 += 3*x - y <= 3 + M*(1-z), "second_constraint"

# Solve and display results
print_solution(prob2)
print(prob2)

## Example 3: Scheduling Problem

Let's consider a scheduling problem where we need to assign employees to workdays.

### Problem description:
- We have three employees: Anna, Kate, and Peter
- Each employee can work up to 3 days per week
- Daily rates are: Anna ($150), Kate ($160), Peter ($140)
- Anna can't work on Monday, Peter can't work on Thursday and Friday
- On Tuesday we need 2 employees, on other days we need 1 employee
- Goal: minimize the total cost

This is a perfect application for integer programming with binary variables.

In [ ]:
prob3 = LpProblem("Scheduling_Problem", LpMinimize)

# Define data
days = ["mon", "tue", "wed", "thu", "fri"]
employees = ["Anna", "Kate", "Peter"]
costs = [150, 160, 140]  # daily rates

# Create binary decision variables for each employee on each day
schedule = LpVariable.dicts("Schedule", (employees, days), cat="Binary")

In [ ]:
# Objective function: minimize total cost
prob3 += lpSum([costs[i] * lpSum([schedule[employee][day] for day in days])
                for i, employee in enumerate(employees)])

In [ ]:
# Constraint: required number of employees each day
required_employees = [1, 2, 1, 1, 1]  # mon, tue, wed, thu, fri
for day, required in zip(days, required_employees):
    prob3 += lpSum([schedule[employee][day] for employee in employees]) == required

# Constraint: employee availability
prob3 += schedule["Anna"]["mon"] == 0  # Anna can't work Monday
prob3 += schedule["Peter"]["thu"] == 0  # Peter can't work Thursday
prob3 += schedule["Peter"]["fri"] == 0  # Peter can't work Friday

# Constraint: maximum workdays per employee
for employee in employees:
    prob3 += lpSum([schedule[employee][day] for day in days]) <= 3

In [ ]:
# Solve and print the solution
print_solution(prob3)

NameError: name 'prob3' is not defined

# Excercise 1

1) Extend the example to four Emploeeys: there is also Mark, who can work on any day and his rate is the lowest: $100 per day but he can work up to 4 days per week.

2) Add a constraint(s) that Mark can work only if Anna is not there (use logical constraint)

3) Bonus:  Make the program interactive: add a checkbox to show which employees are available (and which are not)


In [2]:
!pip install pulp
from pulp import *
import ipywidgets as widgets
from IPython.display import display

# Data
days = ["mon", "tue", "wed", "thu", "fri"]
employees = ["Anna", "Kate", "Peter", "Mark"]
costs = {"Anna": 150, "Kate": 160, "Peter": 140, "Mark": 100}
max_workdays = {"Anna": 3, "Kate": 3, "Peter": 3, "Mark": 4}
required_employees = [1, 2, 1, 1, 1]  # per day

hard_unavailable = {
    "Anna": ["mon"],
    "Peter": ["thu", "fri"],
    "Kate": [],
    "Mark": [],
}

availability_widgets = {
    emp: {
        day: widgets.Checkbox(
            value=(day not in hard_unavailable[emp]),
            description=f"{emp} {day}",
            disabled=(day in hard_unavailable[emp])
        )
        for day in days
    }
    for emp in employees
}


print("Toggle employee availability:")
for emp in employees:
    display(widgets.HBox([availability_widgets[emp][day] for day in days]))

def solve_schedule():
    # Problem definition
    prob = LpProblem("Employee_Scheduling", LpMinimize)

    # Decision variables
    schedule = LpVariable.dicts("Schedule", (employees, days), cat=LpBinary)

    # Objective: Minimize cost
    prob += lpSum([costs[emp] * schedule[emp][day] for emp in employees for day in days])

    # Daily staffing requirement
    for d, day in enumerate(days):
        prob += lpSum([schedule[emp][day] for emp in employees]) == required_employees[d]

    # Employee maximum work days
    for emp in employees:
        prob += lpSum([schedule[emp][day] for day in days]) <= max_workdays[emp]

    # Custom availability (checkboxes)
    for emp in employees:
        for day in days:
            if not availability_widgets[emp][day].value:
                prob += schedule[emp][day] == 0

    # Logical constraint: Mark can only work if Anna is NOT working
    for day in days:
        prob += schedule["Mark"][day] + schedule["Anna"][day] <= 1

    # Solve
    prob.solve()

    # Print solution
    print("\nOptimal Schedule (1 = working):")
    for emp in employees:
        print(f"{emp:5}:", end=" ")
        for day in days:
            print(int(schedule[emp][day].varValue), end=" ")
        print()
    print(f"\nTotal Cost: ${value(prob.objective)}")

# Add a button to solve
solve_button = widgets.Button(description="Solve Schedule")
solve_button.on_click(lambda b: solve_schedule())

display(solve_button)


Toggle employee availability:


Button(description='Solve Schedule', style=ButtonStyle())


Optimal Schedule (1 = working):
Anna : 0 0 1 0 0 
Kate : 1 0 0 1 0 
Peter: 0 1 0 0 0 
Mark : 0 1 0 0 1 

Total Cost: $810.0


## Extension: Hourly Scheduling

We can extend the scheduling problem to assign specific hours rather than just days.

### Revised problem:
- Each employee can work up to 24 hours per week
- Hourly rates: Anna ($15), Kate ($16), Peter ($14)
- Tuesday requires 12 hours of work, other days require 8 hours
- Other constraints remain the same

This requires integer (not just binary) variables to represent hours worked.

In [ ]:
prob4 = LpProblem("Hourly_Scheduling_Problem", LpMinimize)

# Use integer variables for hours (0 to 24)
hours = LpVariable.dicts("Hours", (employees, days), lowBound=0, upBound=24, cat=LpInteger)

# Objective: minimize total cost
hourly_rates = [15, 16, 14]  # hourly rates
prob4 += lpSum([hourly_rates[i] * lpSum([hours[employee][day] for day in days])
               for i, employee in enumerate(employees)])

# Hours required each day
hours_required = [8, 12, 8, 8, 8]  # mon, tue, wed, thu, fri
for day, required in zip(days, hours_required):
    prob4 += lpSum([hours[employee][day] for employee in employees]) == required

# Availability constraints
prob4 += hours["Anna"]["mon"] == 0  # Anna can't work Monday
prob4 += hours["Peter"]["thu"] == 0  # Peter can't work Thursday
prob4 += hours["Peter"]["fri"] == 0  # Peter can't work Friday

# Maximum hours per week
for employee in employees:
    prob4 += lpSum([hours[employee][day] for day in days]) <= 24

# Solve and print the solution
print_solution(prob4)

IndexError: list index out of range

In [8]:
# Excercise 2
# Extend the example to four Emploeeys: there is also Mark, who can work on any day and his rate is the lowest: $100 per day but he can work up to 4 days per week.
# Add a constraint(s) that Mark can work 5h

!pip install pulp
from pulp import *

def print_solution(p):
    p.solve()
    print("Status:", LpStatus[p.status])
    print("\n--- Daily Schedule ---")

    # Gather and format results by day
    schedule = {day: [] for day in days}
    for e in employees:
        for d in days:
            var = hours[e][d]
            if var.varValue is not None and var.varValue > 0:
                schedule[d].append(f"{e} - {int(var.varValue)}h")

    for d in days:
        print(f"{d.capitalize()}:")
        if schedule[d]:
            for entry in schedule[d]:
                print(f"  {entry}")
        else:
            print("  No one scheduled")

    print("\nTotal cost =", value(p.objective))

prob5 = LpProblem("Hourly_Scheduling_Problem_with_Mark", LpMinimize)

employees = ["Anna", "Kate", "Peter", "Mark"]
days = ["mon", "tue", "wed", "thu", "fri"]

hourly_rates = {"Anna": 15, "Kate": 16, "Peter": 14, "Mark": 10}
hours_required = {"mon": 8, "tue": 12, "wed": 8, "thu": 8, "fri": 8}

# Variables: hours worked per day by each employee
hours = LpVariable.dicts("Hours", (employees, days), lowBound=0, upBound=24, cat=LpInteger)

# Binary variables to track whether each employee works each day
works_day = LpVariable.dicts("Works", (employees, days), cat=LpBinary)
M = 24  # Maximum hours per day for linking binary vars

# Objective: minimize total cost
prob5 += lpSum(hourly_rates[e] * hours[e][d] for e in employees for d in days)

# Ensure the total hours per day match requirements
for d in days:
    prob5 += lpSum(hours[e][d] for e in employees) == hours_required[d], f"Total_Hours_{d}"

# Employee day limits and weekly hours
for e in ["Anna", "Kate", "Peter"]:
    prob5 += lpSum(works_day[e][d] for d in days) <= 3, f"{e}_Max_Days"
    prob5 += lpSum(hours[e][d] for d in days) <= 24, f"{e}_Max_Hours"
    for d in days:
        prob5 += hours[e][d] <= M * works_day[e][d], f"{e}_WorkDay_Link_{d}"

# Availability constraints
prob5 += hours["Anna"]["mon"] == 0, "Anna_Not_Monday"
prob5 += hours["Peter"]["thu"] == 0, "Peter_Not_Thursday"
prob5 += hours["Peter"]["fri"] == 0, "Peter_Not_Friday"

# Mark can work only 5 total hours and max 4 days
prob5 += lpSum(hours["Mark"][d] for d in days) == 5, "Mark_Total_5_Hours"
prob5 += lpSum(works_day["Mark"][d] for d in days) <= 4, "Mark_Max_4_Days"
for d in days:
    prob5 += hours["Mark"][d] <= M * works_day["Mark"][d], f"Mark_WorkDay_Link_{d}"

# Mark can only work if Anna is NOT working that day
for d in days:
    prob5 += works_day["Mark"][d] + works_day["Anna"][d] <= 1, f"Mark_Anna_Not_Together_{d}"

for d in days:
    required = 2 if d == "tue" else 1
    prob5 += lpSum(works_day[e][d] for e in ["Anna", "Kate", "Peter", "Mark"]) == required, f"Employee_Count_{d}"

# Solve
print_solution(prob5)


Status: Optimal

--- Daily Schedule ---
Mon:
  Peter - 8h
Tue:
  Peter - 7h
  Mark - 5h
Wed:
  Peter - 8h
Thu:
  Anna - 8h
Fri:
  Anna - 8h

Total cost = 612.0


## Conclusion

Integer programming is a powerful tool for solving optimization problems with discrete decisions. It's particularly useful in scheduling, where we often need to assign resources (like employees) to specific time slots subject to various constraints.

The key benefits include:
1. Ability to model logical conditions (AND, OR, IF-THEN)
2. Natural representation of indivisible resources
3. Optimal solutions for complex constraint satisfaction problems

However, integer programming problems can be computationally intensive as the number of variables increases.